<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/8.%20%EC%9E%90%EC%97%B0%EC%96%B4%20%EC%B2%98%EB%A6%AC%EC%9D%98%20%EC%A0%84%EC%B2%98%EB%A6%AC/01.%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC%EC%A0%84%EC%B2%98%EB%A6%AC%20%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **01. 자연어 처리 전처리 이해하기**
--- 
* 일반적으로 **토큰화, 단어 집합 생성, 정수 인코딩, 패딩, 벡터화의 과정이 있음. 

## 1. 토큰화 (Tokenization) 
* 주어진 텍스트를 단어 또는 문자 단위로 자르는 것 
* 영어의 경우 `spaCy`와 `NLTK`가 있음, 기본적으로 파이썬 `spli`으로도 토큰화할 수 있음. 

### 1. spaCy 사용하기 

In [ ]:
en_text = "A Dog Run back corner near spare bedrooms"

In [ ]:
import spacy
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize(en_text):
    return [tok.text for tok in spacy_en.tokenizer(en_text)]

In [ ]:
print(tokenize(en_text))

### 2. NLTK 사용하기

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize
      (en_text))

### 3. 띄어쓰기로 토큰화

In [ ]:
print(en_text.split())

### 4. 한국어 띄어쓰기 토큰화 

In [ ]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사왔어"

In [ ]:
print(kor_text.split())

* `사과`라는 단어가 4번 등장하는데 각각 다른 조사와 붙어서 등장함. 그럼 기계는 전부 다른 단어로 인식함. 

### 5. 형태소 토큰화 
* 한국어는 보통 형태소 분석기로 토큰화를 진행함. 그 중 `mecab`을 사용해서 시도하겠음.

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

* 위와 같이 조사가 모두 분리된 것을 볼 수 있음. 
### 6. 문자 토큰화

In [ ]:
print(list(en_text))

# 2. 단어 집합 생성 
--- 
단어 집합(vocabulary)란 **중복을 제거한** 텍스트의 총 단어의 집합을 의미함. 

In [ ]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt') # 데이터프레임에 저장
data[:10]

In [ ]:
print(f"전체 샘플의 수: {len(data)}")

In [ ]:
sample_data = data[:100] # 임의로 100개 저장 

In [ ]:
sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-하-|가-힣 ]", "") # 한글과 공백을 제외하고 모두 제거
sample_data[:10]

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
tokenizer = Mecab()

tokenized = []
for sentence in sample_data['document']:
    tmp = tokenizer.morphs(sentence) # 토큰화 
    tmp = [word for word in tmp if not word in stopwords] # 불용어 제거
    tokenized.append(tmp)

In [ ]:
print(tokenized[:10])

In [ ]:
vocab = FreqDist(np.hstack(tokenized))
print(f'단어 집합의 크기 : {len(vocab)}')

In [ ]:
vocab

In [ ]:
vocab["ㅋㅋ"]

In [ ]:
vocab_size = 500
# 상위 vocab_size 개의 단어만 보존 
vocab = vocab.most_common(vocab_size)
print(f'단어 집합의 크기 {len(vocab)}')

# 3. 각 단어에 고유한 정수 부여
--- 
`enumerate()`는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 리턴함 

In [ ]:
# 단어 집합의 단어들에 고유한 index 부여 
word_to_index = {word[0]: index + 2 for index, word in enumerate(vocab)}
word_to_index['pad'] = 1
word_to_index['unk'] = 0

In [ ]:
# tokenized된 문장에서 각 word에 해당하는 index 부여 
encoded = []
for line in tokenized: # 입력 데이터에서 1줄씩 문장을 읽음
    tmp = []
    for w in line: # 각 줄에서 1개씩 
        try:
            tmp.append(word_to_index[w])
        except:
            tmp.append(word_to_index['unk']) # 단어 집합에 없는 단어의 경우 unk로 대체 

        encoded.append(tmp)

In [ ]:
print(encoded[:10])

# 4. 길이가 다른 문장들을 모두 동일한 길이로 바꿔주는 패딩 (padding)
--- 
* 각 문장들을 모두 동일한 길이로 바꿔주는 패딩 작업 진행
* 패딩작업이란 정해준 길이로 모든 샘플들의 길이를 맞춰주되, 정해준 길이보다 짧은 샘플들에는 `pad` 토큰을 추가하여 길이를 맞춰줌 

In [ ]:
max_len = max(len(l) for l in encoded)
min_len = min(len(l) for l in encoded)
avg_len = (sum(map(len, encoded))/len(encoded))

In [ ]:
print('리뷰의 최대 길이 : ', max_len)
print('리뷰의 최소 길이 : ', min_len)
print('리뷰의 평균 길이 : ', avg_len)

In [ ]:
plt.hist([len(s) for s in encoded], bins = 50)
plt.xlabel('length of sample')
plt.ylabel('number of sample')
plt.show()

In [ ]:
for line in encoded:
    if len(line) < max_len:
        line += [word_to_index['pad']] * (max_len - len(line)) # 나머지는 전부 pad 토큰으로 채움 

In [ ]:
print(encoded[:3])

* 단어들을 고유한 정수로 맵핑하였음. 각 정수를 고유한 **단어 벡터**로 바꾸는 작업이 필요하며 이때는 원-핫 인코딩, 워드 임베딩 등의 방식이 있음.  